# Add timezone to HYDAT daily flow series

WSC provides daily average flows calculated in Local Standard Time, but does not provide easily accessible information on what LST for each gauge is. We need to somehow map the gauges onto time zones to find the right time coordinates for storing the daily average flows into netcdf. 

Communication with WSC (2023-04-07) indicates that:
```
Q: According to this wikipedia page (https://en.wikipedia.org/wiki/Time_in_Canada) Local Standard Time seems to be a regional choice, and a single province/territory could have regions in it with different time zones. Could you tell me if these daily averages are calculated using a single LST choice per province/territory, or if these daily averages are based on the exact time zone the region the station is in uses? 

A: Our Field Techs use the official time zone for the area they are in.  We do not use municipal or any type of local or municipal exceptions. [..] Here is a link to the Government of Canada information page on time zones in Canada.  I would suggest you use that one as that is what we would follow.
```
https://nrc.canada.ca/en/certifications-evaluations-standards/canadas-official-time/time-zones-daylight-saving-time

To address this, we created a shapefile that shows Standard Time for the United States and Canada. See: `'../0_standard_timezone_shapefile`.

Here we map the locations of Canadian gauges onto this shapefile to find out what Standard Time means for each individual gauge. This is stored in the CAMELS-spat metadata file.

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [3]:
# Get the required info from the config file
data_path = cs.read_from_config(config_file,'data_path')

# CAMELS-spat metadata
cs_meta_path = cs.read_from_config(config_file,'cs_basin_path')
cs_meta_name = cs.read_from_config(config_file,'cs_meta_name')
cs_unusable_name = cs.read_from_config(config_file,'cs_unusable_name')

# Standard time shapefile
code_path = cs.read_from_config(config_file,'code_path')
tz_shp_path = cs.read_from_config(config_file,'standard_time_shp_path')
tz_shp_name = cs.read_from_config(config_file,'standard_time_shp_name')

### Data loading

In [4]:
# CAMELS-spat metadata file
cs_meta_path = Path(data_path) / cs_meta_path
cs_meta = pd.read_csv(cs_meta_path / cs_meta_name)

In [5]:
# Open list of unusable stations; Enforce reading IDs as string to keep leading 0's
cs_unusable = pd.read_csv(cs_meta_path / cs_unusable_name, dtype={'Station_id': object}) 

In [6]:
# Timezone shapefile
tz_shp_path = Path(code_path) / tz_shp_path
tz_shp = gpd.read_file(tz_shp_path / tz_shp_name)

### Loop over sites and map location onto timezones

In [7]:
# Subset the cs_unusable list to those entries that indicate station does not have IV values
subset_column = 'Missing'
subset_to = 'dv'
cs_unusable_masked = cs_unusable[cs_unusable[subset_column] == subset_to]

In [8]:
# Prepare the metadata file
meta_column_tz = 'dv_flow_obs_timezone'
c_tz = np.where(cs_meta.columns == meta_column_tz)[0][0]

In [9]:
# Loop over the Canada stations only
for ix,row in cs_meta.iterrows():
    if row.Country == 'CAN':
        
        # Skip to next if we didn't have any usable daily values here
        if row.Station_id in cs_unusable_masked['Station_id'].values:
            continue # to next row
        
        # Get station latitude and longitude
        station = Point(row.Station_lon, row.Station_lat)
        
        # Find the timezone
        for shp_ix,shp_row in tz_shp.iterrows():
            if station.within(shp_row['geometry']):
                break
        print(f'{ix:04d}. Station {row.Station_id} located in timezone {shp_row.tz}')
        
        # Store the timezone in the metadata file
        cs_meta.iat[ix,c_tz] = shp_row.tz

0000. Station 01AD002 located in timezone AST
0001. Station 01AD003 located in timezone AST
0002. Station 01AE001 located in timezone EST
0003. Station 01AF007 located in timezone AST
0004. Station 01AF009 located in timezone AST
0005. Station 01AJ003 located in timezone AST
0006. Station 01AJ004 located in timezone AST
0007. Station 01AJ010 located in timezone AST
0008. Station 01AK001 located in timezone AST
0009. Station 01AK006 located in timezone AST
0010. Station 01AK007 located in timezone AST
0011. Station 01AL002 located in timezone AST
0012. Station 01AL004 located in timezone AST
0013. Station 01AM001 located in timezone AST
0014. Station 01AN002 located in timezone AST
0015. Station 01AP002 located in timezone AST
0016. Station 01AP004 located in timezone AST
0017. Station 01AP006 located in timezone AST
0018. Station 01AQ001 located in timezone AST
0019. Station 01BC001 located in timezone AST
0020. Station 01BD008 located in timezone EST
0021. Station 01BE001 located in t

0184. Station 02LC027 located in timezone EST
0185. Station 02LC043 located in timezone EST
0186. Station 02LD005 located in timezone EST
0187. Station 02LG005 located in timezone EST
0188. Station 02MB006 located in timezone EST
0189. Station 02MC001 located in timezone EST
0190. Station 02MC026 located in timezone EST
0191. Station 02MC028 located in timezone EST
0192. Station 02NE011 located in timezone EST
0193. Station 02NF003 located in timezone EST
0194. Station 02OA057 located in timezone EST
0195. Station 02OB032 located in timezone EST
0196. Station 02OB037 located in timezone EST
0197. Station 02OD003 located in timezone EST
0198. Station 02OE027 located in timezone EST
0199. Station 02OE032 located in timezone EST
0200. Station 02OG026 located in timezone EST
0201. Station 02OJ024 located in timezone EST
0202. Station 02OJ026 located in timezone EST
0203. Station 02PA007 located in timezone EST
0204. Station 02PB006 located in timezone EST
0205. Station 02PC018 located in t

0373. Station 05CD006 located in timezone MST
0374. Station 05CD007 located in timezone MST
0375. Station 05CE002 located in timezone MST
0376. Station 05CE006 located in timezone MST
0377. Station 05CE010 located in timezone MST
0378. Station 05CE011 located in timezone MST
0379. Station 05CE018 located in timezone MST
0380. Station 05CE020 located in timezone MST
0381. Station 05CG004 located in timezone MST
0382. Station 05CG006 located in timezone MST
0383. Station 05CK001 located in timezone MST
0384. Station 05CK005 located in timezone MST
0385. Station 05DA007 located in timezone MST
0386. Station 05DA009 located in timezone MST
0387. Station 05DA010 located in timezone MST
0388. Station 05DB002 located in timezone MST
0389. Station 05DB005 located in timezone MST
0390. Station 05DB006 located in timezone MST
0391. Station 05DC006 located in timezone MST
0392. Station 05DC011 located in timezone MST
0393. Station 05DC012 located in timezone MST
0394. Station 05DD004 located in t

0553. Station 05SD003 located in timezone CST
0554. Station 05TD001 located in timezone CST
0555. Station 05TE002 located in timezone CST
0556. Station 05TF002 located in timezone CST
0557. Station 05TG002 located in timezone CST
0558. Station 05TG003 located in timezone CST
0559. Station 05TG006 located in timezone CST
0560. Station 05UA003 located in timezone CST
0561. Station 05UF004 located in timezone CST
0562. Station 05UG001 located in timezone CST
0563. Station 05UH001 located in timezone CST
0564. Station 05UH002 located in timezone CST
0565. Station 06AA001 located in timezone MST
0566. Station 06AB001 located in timezone MST
0567. Station 06AB002 located in timezone MST
0568. Station 06AD001 located in timezone CST
0569. Station 06AD006 located in timezone MST
0570. Station 06AD008 located in timezone CST
0571. Station 06AD010 located in timezone CST
0572. Station 06AD011 located in timezone CST
0573. Station 06AF001 located in timezone CST
0574. Station 06AG001 located in t

0754. Station 08EE004 located in timezone PST
0755. Station 08EE008 located in timezone PST
0756. Station 08EE012 located in timezone PST
0757. Station 08EE013 located in timezone PST
0758. Station 08EE020 located in timezone PST
0759. Station 08EE025 located in timezone PST
0760. Station 08EF001 located in timezone PST
0761. Station 08EF005 located in timezone PST
0762. Station 08EG012 located in timezone PST
0763. Station 08FA002 located in timezone PST
0764. Station 08FB004 located in timezone PST
0765. Station 08FB006 located in timezone PST
0766. Station 08FC003 located in timezone PST
0767. Station 08FE003 located in timezone PST
0768. Station 08FF001 located in timezone PST
0769. Station 08FF002 located in timezone PST
0770. Station 08FF003 located in timezone PST
0771. Station 08FF006 located in timezone PST
0772. Station 08GA071 located in timezone PST
0773. Station 08GA072 located in timezone PST
0774. Station 08GA075 located in timezone PST
0775. Station 08GA077 located in t

0954. Station 09CA006 located in timezone MST
0955. Station 09CB001 located in timezone MST
0956. Station 09CD001 located in timezone MST
0957. Station 09DB001 located in timezone MST
0958. Station 09DD003 located in timezone MST
0959. Station 09DD004 located in timezone MST
0960. Station 09EA003 located in timezone MST
0961. Station 09EA004 located in timezone MST
0962. Station 09EB003 located in timezone MST
0963. Station 09ED001 located in timezone LST
0964. Station 09FB002 located in timezone MST
0965. Station 09FC001 located in timezone MST
0966. Station 09FD002 located in timezone MST
0967. Station 10AA001 located in timezone MST
0968. Station 10AA004 located in timezone MST
0969. Station 10AA005 located in timezone MST
0970. Station 10AB001 located in timezone MST
0971. Station 10AC005 located in timezone PST
0972. Station 10BE001 located in timezone MST
0973. Station 10BE004 located in timezone MST
0974. Station 10BE007 located in timezone MST
0975. Station 10BE009 located in t

In [10]:
# Save the metadata file
cs_meta.to_csv(cs_meta_path / cs_meta_name, encoding='utf-8', index=False)